# Homework 6

Building monthly cash flow model with 

Assets: 100MM collateral paying 7%
Liabilities: 75MM class A paying 6%, 15mm class B paying 9%, 10MM class C.

In [1]:
import pandas as pd

# Set the display precision to 2 decimal places
pd.set_option('display.precision', 2)


# Summary

Without OC test, 

at 0% annualized default rate, losses are 0 for all tranches and 

    Accumulated Interest for each tranche:
    Tranche A: 45000000.00
    Tranche B: 13500000.00
    Tranche EQUITY: 11500000.00
    
at 1% annualized default rate, 

    losses are {'EQUITY': 9561792.499119848, 'B': 0.0, 'A': 0.0}
    Accumulated Interest for each tranche:
    Tranche A: 45000000.00
    Tranche B: 13500000.00
    Tranche EQUITY: 8069439.51
    
The default rate that results in a 1 million loss for tranche B is: 0.011585,
    
    {'EQUITY': 10000000.0, 'B': 1000000.0, 'A': 0.0}
    Accumulated Interest for each tranche:
    Tranche A: 45000000.00
    Tranche B: 13500000.00
    Tranche EQUITY: 7543162.91
    
The default rate that results in a 1 million loss for tranche A is: 0.029662,

    losses are {'EQUITY': 10000000.0, 'B': 15000000.0, 'A': 1000000.0}
    Accumulated Interest for each tranche:
    Tranche A: 45000000.00
    Tranche B: 12091223.89
    Tranche EQUITY: 3276987.02
    
With OC test,

at 0% annualized default rate, everything is the same as without OC test.

at 1% annualized default rate, accumulated interest is different than without OC test.

    losses are {'EQUITY': 9561792.499119848, 'B': 0.0, 'A': 0.0}
    Accumulated Interest for each tranche:
    Tranche A: 45000000.00
    Tranche B: 12839463.72
    Tranche EQUITY: 5328577.72
    
The default rate that results in a 1 million loss for tranche B is: 0.011585, which is the same as without OC test, but the accumulated interest is different.

    losses are {'EQUITY': 10000000.0, 'B': 1000000.0, 'A': 0.0}
    Accumulated Interest for each tranche:
    Tranche A: 44973120.34
    Tranche B: 12105421.71
    Tranche EQUITY: 4607176.29
   
The default rate that results in a 1 million loss for tranche A is: 0.02816, and both default rate and accumulated interest are different than without OC test.

    losses are {'EQUITY': 10000000.0, 'B': 15000000.0, 'A': 1000000.0000005364}
    Accumulated Interest for each tranche:
    Tranche A: 42910305.68
    Tranche B: 4538235.11
    Tranche EQUITY: 1744887.70

    
    







## Problem 1

Build model without an OC test with the following scenarios
    
0% annualized default rate

1% annualized default rate (solve for the monthly default rate)

Find the annualized default rate that causes the B class to lose ~1MM of principal

Find the annualized default rate that causes the A class to lose ~1MM of principal

The following classes are defined for convenience and logic flow embedding in the CLO model.

Enum class ensures the seniority of the tranches.

In [2]:
from typing import List
from collections import deque
from enum import Enum, auto


class TrancheType(Enum):
    A = -1
    B = -2
    EQUITY = -3

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.value == other.value

    def __ne__(self, other):
        return self.value != other.value

    def __gt__(self, other):
        return self.value > other.value

    def __lt__(self, other):
        return self.value < other.value

    def __ge__(self, other):
        return self.value >= other.value

    def __le__(self, other):
        return self.value <= other.value


class Tranche:

    def __init__(self, notional_balance, interest_rate, class_type, trigger=None):
        self.t = 0
        self.og_notional_balance = notional_balance
        self.notional_balance = notional_balance
        self.accumulated_interest = 0
        self.interest_rate = interest_rate
        self.class_type = class_type
        self.trigger = trigger
        self.oc_ratio = None
        self.state = {}

    def __str__(self):
        return f"Tranche({self.notional_balance}, {self.interest_rate}, {self.class_type})"

    def __repr__(self):
        return self.__str__()

    def set_trigger(self, trigger):
        self.trigger = trigger

    def check_oc_test(self, **kwargs) -> bool:
        if self.trigger is None:
            return True
        """return True if OC test is passed, False otherwise."""
        self._update_oc_ratio(**kwargs)
        return self.oc_ratio >= self.trigger

    def _update_oc_ratio(self, **kwargs):
        # Use Not’l After Defaults / A Class Not’l for OC Ratio
        notional_after_defaults = kwargs.get("notional_after_defaults")
        upper_tranches = kwargs.get("upper_tranches")
        upper_tranche_notional_list = [tranche.notional_balance for tranche in upper_tranches]
        total_notional = sum(upper_tranche_notional_list) + self.notional_balance
        self.oc_ratio = notional_after_defaults / total_notional

    def inject_interest(self, value) -> float:
        self.t += 1
        scheduled_interest_payment = self.calculate_interest()
        if self.class_type == TrancheType.EQUITY:
            scheduled_interest_payment = None
            actual_interest_payment = value
        else:
            actual_interest_payment = min(scheduled_interest_payment, value)
        self._log_state(f"Scheduled Interest Payment", scheduled_interest_payment)
        self._log_state("Actual Interest Payment", actual_interest_payment)
        self.accumulated_interest += actual_interest_payment
        self._log_state("Notional Balance", self.notional_balance)
        self._log_state("trigger", self.trigger)
        self._log_state("test_ratio", self.oc_ratio)
        residual = value - actual_interest_payment
        return residual

    def _log_state(self, param, value):
        if value is None:
            return
        if self.t not in self.state:
            self.state[self.t] = {}
        self.state[self.t][param + f" - Class {self.class_type.name}"] = value

    def calculate_interest(self) -> float:
        return self.notional_balance * self.interest_rate / 12


class CLO:

    def __init__(self, asset_par, interest_rate):
        self.t = 0  # month index
        self.tranches: List[Tranche] = []
        self.starting_asset_par = asset_par  # current value of the assets
        self.asset_par = asset_par  # current value of the assets
        self.interest_rate = interest_rate  # annualized; we should be calculating monthly
        self.default_rate = 0
        self.monthly_default_rate = 0
        self.state = {}

    def __str__(self):
        return f"CLO({sorted(self.tranches, key=lambda x: x.class_type, reverse=True)}) with {self.default_rate} default rate"

    def __repr__(self):
        return self.__str__()

    def get_sorted_tranches(self, senior_first=True):
        return sorted(self.tranches, key=lambda x: x.class_type, reverse=senior_first)

    def set_default_rate(self, default_rate):
        self.default_rate = default_rate  # annualized; we should be calculating monthly
        self.monthly_default_rate = self.calculate_monthly_default_rate(default_rate)

    def add_tranche(self, *tranches):
        for tranche in tranches:
            self.tranches.append(tranche)

    def receive_interest(self, value):
        upper_tranches = []
        for tranche in self.get_sorted_tranches():
            value = tranche.inject_interest(value)
            _has_passed_oc_test = tranche.check_oc_test(notional_after_defaults=self.asset_par, upper_tranches=upper_tranches)
            if not _has_passed_oc_test:
                # if OC test fails, all residual interest payments go to the current tranche principal
                tranche.notional_balance -= min(value, tranche.notional_balance)
                return
            # if OC test passes, we continue to the next tranche
            upper_tranches.append(tranche)

    def calculate_monthly_interest(self):
        self.t += 1
        if self.asset_par <= 0:
            return 0
        self._defaulting()
        interest_payment = self.asset_par * self.interest_rate / 12
        self._log_state("Interest Payment", interest_payment)
        return interest_payment

    def _defaulting(self):
        self._log_state("Notional Balance Before Default", self.asset_par)
        self.asset_par = self.asset_par * (1 - self.monthly_default_rate)
        self._log_state("Notional Balance", self.asset_par)

    def _log_state(self, param, value):
        if self.t not in self.state:
            self.state[self.t] = {}
        self.state[self.t][param] = value

    def calculate_loss(self):
        reversed_tranches = self.get_sorted_tranches(senior_first=False)
        total_loss = self.starting_asset_par - self.asset_par
        loss_result = {}
        for tranche in reversed_tranches:
            loss = min(tranche.og_notional_balance, total_loss)
            total_loss -= loss
            loss_result[tranche.class_type.name] = loss
        return loss_result

    @staticmethod
    def calculate_monthly_default_rate(annualized_default_rate):
        return 1 - (1 - annualized_default_rate) ** (
                1 / 12)  # (1 - monthly default rate) ** 12 = 1 - annualized default rate -> monthly default rate = 1 - (1 - annualized default rate) ** (1/12)

    def generate_result_status(self):
        dfs = []
        dfs.append(pd.DataFrame(self.state).T)
        for tranche in self.tranches:
            dfs.append(pd.DataFrame(tranche.state).T)
        return pd.concat(dfs, axis=1)

## a) 0% annualized default rate

In [3]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a, b, c)
clo.set_default_rate(0.)

clo

CLO([Tranche(75000000.0, 0.06, A), Tranche(15000000.0, 0.09, B), Tranche(10000000.0, 0, EQUITY)]) with 0.0 default rate

In [4]:
for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)

In [5]:
import pandas as pd

result_df = clo.generate_result_status()
result_df

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
2,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
3,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
4,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
5,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...
116,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
117,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
118,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07
119,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95833.33,1.00e+07


In [6]:
clo.calculate_loss()

{'EQUITY': 0.0, 'B': 0.0, 'A': 0.0}

In [7]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 13500000.00
Tranche EQUITY: 11500000.00


## b) 1% annualized default rate

In [8]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a, b, c)
clo.set_default_rate(0.01)

clo

CLO([Tranche(75000000.0, 0.06, A), Tranche(15000000.0, 0.09, B), Tranche(10000000.0, 0, EQUITY)]) with 0.01 default rate

In [9]:
for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)

In [10]:
result_df = clo.generate_result_status()
result_df

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,9.99e+07,582844.98,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,95344.98,1.00e+07
2,9.99e+07,9.98e+07,582357.04,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,94857.04,1.00e+07
3,9.98e+07,9.97e+07,581869.50,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,94369.50,1.00e+07
4,9.97e+07,9.97e+07,581382.37,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,93882.37,1.00e+07
5,9.97e+07,9.96e+07,580895.65,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,93395.65,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...
116,9.08e+07,9.07e+07,529326.55,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,41826.55,1.00e+07
117,9.07e+07,9.07e+07,528883.41,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,41383.41,1.00e+07
118,9.07e+07,9.06e+07,528440.64,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,40940.64,1.00e+07
119,9.06e+07,9.05e+07,527998.24,375000.0,375000.0,7.50e+07,112500.0,112500.0,1.50e+07,40498.24,1.00e+07


In [11]:
clo.calculate_loss()

{'EQUITY': 9561792.499119848, 'B': 0.0, 'A': 0.0}

In [12]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 13500000.00
Tranche EQUITY: 8069439.51


## c) Find the annualized default rate that causes the B class to lose ~1MM of principal

In [13]:
def objective_function(default_rate, target_loss, target_class):
    _clo = CLO(asset_par=100e6, interest_rate=0.07)
    _a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A)
    _b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B)
    _c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
    _clo.add_tranche(_a, _b, _c)
    _clo.set_default_rate(default_rate)
    for i in range(120):
        _monthly_interest = _clo.calculate_monthly_interest()
        _clo.receive_interest(_monthly_interest)
    loss = _clo.calculate_loss()
    return loss[target_class.name] - target_loss

In [14]:
from scipy.optimize import bisect

result = bisect(objective_function, 0.01, 0.5, args=(1e6, TrancheType.B), xtol=1e-12)

print("The default rate that results in a $1 million loss for tranche B is:", result)


The default rate that results in a $1 million loss for tranche B is: 0.011585743964697033


In [15]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a)
clo.add_tranche(b)
clo.add_tranche(c)
clo.set_default_rate(result)

for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)

{x: round(y, 2) for x, y in clo.calculate_loss().items()}

{'EQUITY': 10000000.0, 'B': 1000000.0, 'A': 0.0}

In [16]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 13500000.00
Tranche EQUITY: 7543162.91


## d) Find the annualized default rate that causes the A class to lose ~1MM of principal

In [17]:
result = bisect(objective_function, 0.01, 0.8, args=(1e6, TrancheType.A), xtol=1e-12)

print("The default rate that results in a $1 million loss for tranche A is:", result)

The default rate that results in a $1 million loss for tranche A is: 0.029661703761548783


In [18]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a, b, c)
clo.set_default_rate(result)

In [19]:
for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)
{x: round(y, 2) for x, y in clo.calculate_loss().items()}

{'EQUITY': 10000000.0, 'B': 15000000.0, 'A': 1000000.0}

In [20]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 12091223.89
Tranche EQUITY: 3276987.02


## Problem 2 - Building model with OC test

Build model with OC test with the same scenarios as above.

## a) 0% annualized default rate

In [21]:
def objective_function_with_trigger(default_rate, target_loss, target_class):
    _clo = CLO(asset_par=100e6, interest_rate=0.07)
    _a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A, trigger = 1.2)
    _b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B, trigger = 1.05)
    _c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
    _clo.add_tranche(_a, _b, _c)
    _clo.set_default_rate(default_rate)
    for i in range(120):
        _monthly_interest = _clo.calculate_monthly_interest()
        _clo.receive_interest(_monthly_interest)
    loss = _clo.calculate_loss()
    return loss[target_class.name] - target_loss

In [22]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A, trigger = 1.2)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B, trigger = 1.05)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.set_default_rate(0.)
clo.add_tranche(a, b, c)

In [23]:
for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)

In [24]:
result_df = clo.generate_result_status()
result_df

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,trigger - Class A,test_ratio - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,trigger - Class B,test_ratio - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,NaN,112500.0,112500.0,1.50e+07,1.05,NaN,95833.33,1.00e+07
2,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
3,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
4,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
5,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
117,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
118,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07
119,1.00e+08,1.00e+08,583333.33,375000.0,375000.0,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,95833.33,1.00e+07


In [25]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 13500000.00
Tranche EQUITY: 11500000.00


## b) 1% annualized default rate


In [26]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A, trigger = 1.2)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B, trigger = 1.05)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.set_default_rate(0.01)
clo.add_tranche(a, b, c)

for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)

In [27]:
result_df = clo.generate_result_status()

In [28]:
result_df

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,trigger - Class A,test_ratio - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,trigger - Class B,test_ratio - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,9.99e+07,582844.98,375000.0,375000.0,7.50e+07,1.2,NaN,112500.00,112500.00,1.50e+07,1.05,NaN,95344.98,1.00e+07
2,9.99e+07,9.98e+07,582357.04,375000.0,375000.0,7.50e+07,1.2,1.33,112500.00,112500.00,1.50e+07,1.05,1.11,94857.04,1.00e+07
3,9.98e+07,9.97e+07,581869.50,375000.0,375000.0,7.50e+07,1.2,1.33,112500.00,112500.00,1.50e+07,1.05,1.11,94369.50,1.00e+07
4,9.97e+07,9.97e+07,581382.37,375000.0,375000.0,7.50e+07,1.2,1.33,112500.00,112500.00,1.50e+07,1.05,1.11,93882.37,1.00e+07
5,9.97e+07,9.96e+07,580895.65,375000.0,375000.0,7.50e+07,1.2,1.33,112500.00,112500.00,1.50e+07,1.05,1.11,93395.65,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,9.08e+07,9.07e+07,529326.55,375000.0,375000.0,7.50e+07,1.2,1.21,89426.59,89426.59,1.19e+07,1.05,1.04,NaN,NaN
117,9.07e+07,9.07e+07,528883.41,375000.0,375000.0,7.50e+07,1.2,1.21,88939.84,88939.84,1.19e+07,1.05,1.04,NaN,NaN
118,9.07e+07,9.06e+07,528440.64,375000.0,375000.0,7.50e+07,1.2,1.21,88452.76,88452.76,1.18e+07,1.05,1.04,NaN,NaN
119,9.06e+07,9.05e+07,527998.24,375000.0,375000.0,7.50e+07,1.2,1.21,87965.35,87965.35,1.17e+07,1.05,1.04,NaN,NaN


In [29]:
clo.calculate_loss()

{'EQUITY': 9561792.499119848, 'B': 0.0, 'A': 0.0}

In [30]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 45000000.00
Tranche B: 12839463.72
Tranche EQUITY: 5328577.72


## c) Find the annualized default rate that causes the B class to lose ~1MM of principal

In [31]:
result = bisect(objective_function_with_trigger, 0.01, 0.5, args=(1e6, TrancheType.B), xtol=1e-12)

print("The default rate that results in a $1 million loss for tranche B is:", result)

The default rate that results in a $1 million loss for tranche B is: 0.011585743964697033


In [32]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A, trigger = 1.2)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B, trigger = 1.05)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a, b, c)
clo.set_default_rate(result)


In [33]:
for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)


In [34]:
clo.generate_result_status()

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,trigger - Class A,test_ratio - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,trigger - Class B,test_ratio - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,9.99e+07,582767.12,375000.00,375000.00,7.50e+07,1.2,NaN,112500.0,112500.0,1.50e+07,1.05,NaN,95267.12,1.00e+07
2,9.99e+07,9.98e+07,582201.47,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,94701.47,1.00e+07
3,9.98e+07,9.97e+07,581636.35,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,94136.35,1.00e+07
4,9.97e+07,9.96e+07,581071.79,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,93571.79,1.00e+07
5,9.96e+07,9.95e+07,580507.78,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,93007.78,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,8.94e+07,8.93e+07,521187.27,372013.23,372013.23,7.44e+07,1.2,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,8.93e+07,8.93e+07,520681.38,372013.23,372013.23,7.44e+07,1.2,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,8.93e+07,8.92e+07,520175.99,371269.89,371269.89,7.43e+07,1.2,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,8.92e+07,8.91e+07,519671.08,371269.89,371269.89,7.43e+07,1.2,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN


N/A in the dataframe means there is no cash flow for that month since the test is not passed for the previous tranche.

In [35]:
clo.calculate_loss()

{'EQUITY': 10000000.0, 'B': 999999.9999864548, 'A': 0.0}

In [36]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 44973120.34
Tranche B: 12105421.71
Tranche EQUITY: 4607176.29


## d) Find the annualized default rate that causes the A class to lose ~1MM of principal

In [37]:
result = bisect(objective_function_with_trigger, 0.01, 0.8, args=(1e6, TrancheType.A), xtol=1e-14)
print("The default rate that results in a $1 million loss for tranche A is:", result)

The default rate that results in a $1 million loss for tranche A is: 0.029661703761655438


In [38]:
clo = CLO(asset_par=100e6, interest_rate=0.07)
a = Tranche(notional_balance=75e6, interest_rate=0.06, class_type=TrancheType.A, trigger = 1.2)
b = Tranche(notional_balance=15e6, interest_rate=0.09, class_type=TrancheType.B, trigger = 1.05)
c = Tranche(notional_balance=10e6, interest_rate=0, class_type=TrancheType.EQUITY)
clo.add_tranche(a, b, c)
clo.set_default_rate(result)

for i in range(120):
    monthly_interest = clo.calculate_monthly_interest()
    clo.receive_interest(monthly_interest)


In [39]:
clo.generate_result_status()

,Notional Balance Before Default,Notional Balance,Interest Payment,Scheduled Interest Payment - Class A,Actual Interest Payment - Class A,Notional Balance - Class A,trigger - Class A,test_ratio - Class A,Scheduled Interest Payment - Class B,Actual Interest Payment - Class B,Notional Balance - Class B,trigger - Class B,test_ratio - Class B,Actual Interest Payment - Class EQUITY,Notional Balance - Class EQUITY
1,1.00e+08,9.97e+07,581871.46,375000.00,375000.00,7.50e+07,1.2,NaN,112500.0,112500.0,1.50e+07,1.05,NaN,94371.46,1.00e+07
2,9.97e+07,9.95e+07,580413.26,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,92913.26,1.00e+07
3,9.95e+07,9.93e+07,578958.70,375000.00,375000.00,7.50e+07,1.2,1.33,112500.0,112500.0,1.50e+07,1.05,1.11,91458.70,1.00e+07
4,9.93e+07,9.90e+07,577507.80,375000.00,375000.00,7.50e+07,1.2,1.32,112500.0,112500.0,1.50e+07,1.05,1.10,90007.80,1.00e+07
5,9.90e+07,9.88e+07,576060.52,375000.00,375000.00,7.50e+07,1.2,1.32,112500.0,112500.0,1.50e+07,1.05,1.10,88560.52,1.00e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,7.49e+07,7.47e+07,436021.05,327028.14,327028.14,6.54e+07,1.2,1.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,7.47e+07,7.46e+07,434928.35,326483.17,326483.17,6.53e+07,1.2,1.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,7.46e+07,7.44e+07,433838.40,325940.95,325940.95,6.52e+07,1.2,1.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,7.44e+07,7.42e+07,432751.17,325401.46,325401.46,6.51e+07,1.2,1.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
clo.calculate_loss()

{'EQUITY': 10000000.0, 'B': 15000000.0, 'A': 1000000.0000005364}

In [41]:
# Get accumulated interest for each tranche
print("Accumulated Interest for each tranche:")
for tranche in clo.tranches:
    print(f"Tranche {tranche.class_type.name}: {tranche.accumulated_interest:.2f}")

Accumulated Interest for each tranche:
Tranche A: 42910305.68
Tranche B: 4538235.11
Tranche EQUITY: 1744887.70
